#  IF4140 | EDA

Group Members:
- Muhammad Fadli Alfarizi (13121140)


## Import Libraries

In [65]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from prefixspan import PrefixSpan
from pymining import seqmining

## Import Dataset

In [9]:
df = pd.read_csv('https://drive.google.com/uc?id=1mIMT-kzQiyS5U5gHTA1V70Pvs7ya6dJ8')

df.reset_index(drop=True, inplace=True)

df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['Date'] = df['Date'].astype('int64') // 10 ** 9

df.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,1575849600,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,1575849600,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,1575849600,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,1575849600,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,1575849600,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom


## 1. EDA

In [10]:
df.info()

print(f"\nshape: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  int64  
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 32.7+ MB

shape: (536350, 8)


In [11]:
# check missing values

print(df.isnull().sum())

print(f"\nnumber of missing values: {df.isnull().sum().sum()}")

TransactionNo     0
Date              0
ProductNo         0
ProductName       0
Price             0
Quantity          0
CustomerNo       55
Country           0
dtype: int64

number of missing values: 55


In [12]:
# check number of duplicates

print(f"number of duplicates {df.duplicated().sum()}")

number of duplicates 5200


In [13]:
# check number of cancelled orders based on 'TransactionNo'
print(f"number of cancelled orders: {df['TransactionNo'].str.contains('C').sum()}")

# check number of cancelled orders based on -1 on 'Quantity'
print(f"number of cancelled orders: {df[df['Quantity'] == -1].shape[0]}")


number of cancelled orders: 8585
number of cancelled orders: 3575


In [14]:
"""
There is a transaction with 80k 'Quantity'
"""

df.sort_values('Quantity', ascending=False)

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
142,581483,1575849600,23843,Paper Craft Little Birdie,12.38,80995,16446.0,United Kingdom
473985,541431,1547769600,23166,Medium Ceramic Top Storage Jar,11.32,74215,12346.0,United Kingdom
41367,578841,1574640000,84826,Asstd Design 3d Paper Stickers,6.19,12540,13256.0,United Kingdom
117790,573008,1572134400,84077,World War 2 Gliders Asstd Designs,10.47,4800,12901.0,United Kingdom
331834,554868,1558915200,22197,Popcorn Holder,10.99,4300,13135.0,United Kingdom
...,...,...,...,...,...,...,...,...
378551,C550456,1555545600,21175,Gin And Tonic Diet Metal Sign,12.15,-2000,15749.0,United Kingdom
378552,C550456,1555545600,21108,Fairy Cake Flannel Assorted Colour,12.40,-3114,15749.0,United Kingdom
533232,C536757,1543708800,84347,Rotating Silver Angels T-Light Hldr,10.28,-9360,15838.0,United Kingdom
475332,C541433,1547769600,23166,Medium Ceramic Top Storage Jar,11.32,-74215,12346.0,United Kingdom


In [15]:
"""
Recheck the cancelled orders

It can be observed that the order is cancelled
"""

df[df['Quantity'] < 0].sort_values('Quantity', ascending=True)

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
1616,C581484,1575849600,23843,Paper Craft Little Birdie,6.19,-80995,16446.0,United Kingdom
475332,C541433,1547769600,23166,Medium Ceramic Top Storage Jar,11.32,-74215,12346.0,United Kingdom
533232,C536757,1543708800,84347,Rotating Silver Angels T-Light Hldr,10.28,-9360,15838.0,United Kingdom
378552,C550456,1555545600,21108,Fairy Cake Flannel Assorted Colour,12.40,-3114,15749.0,United Kingdom
378551,C550456,1555545600,21175,Gin And Tonic Diet Metal Sign,12.15,-2000,15749.0,United Kingdom
...,...,...,...,...,...,...,...,...
246353,C562709,1565222400,22061,Large Cake Stand Hanging Strawbery,18.96,-1,17511.0,United Kingdom
246354,C562709,1565222400,21232,Strawberry Ceramic Trinket Pot,11.34,-1,17511.0,United Kingdom
536344,C536548,1543622400,22631,Circus Parade Lunch Box,12.25,-1,12472.0,Germany
1620,C581569,1575849600,84978,Hanging Heart Jar T-Light Holder,6.19,-1,17315.0,United Kingdom


## 2. Data Preprocessing

In [16]:
# remove duplicates value
df_duplicates_removed = df.drop_duplicates()

# remove missing values
df_missing_values_removed = df_duplicates_removed.dropna()

In [17]:
print(f"shape before cancelled orders are removed: {df_missing_values_removed.shape}")

# remove cancelled orders
df_cancelled_orders_removed = df_missing_values_removed[~df_missing_values_removed['TransactionNo'].str.contains('C')]
df_cancelled_orders_removed = df_missing_values_removed[df_missing_values_removed['Quantity'] > 0]

order_cancelled = df_missing_values_removed[df_missing_values_removed['Quantity'] < 0]
order_cancelled.loc[:, 'Quantity'] = order_cancelled['Quantity'] * -1

print(f"shape after cancelled orders are removed (step 1): {df_cancelled_orders_removed.shape}")

columns = ['ProductNo', 'Quantity', 'CustomerNo']

df_merged = pd.merge(df_cancelled_orders_removed, order_cancelled,
                     on=columns,
                     how='inner', suffixes=('_df_cancelled_orders_removed', '_order_cancelled'))

# Step 1: Create tuples for comparison
df_cancelled_orders_removed_tuples = df_cancelled_orders_removed.apply(
    lambda row: (row['TransactionNo'], row['ProductNo'], row['Quantity'], row['CustomerNo']), axis=1
)
df_merged_tuples = df_merged.apply(
    lambda row: (
        row['TransactionNo_df_cancelled_orders_removed'], row['ProductNo'], row['Quantity'], row['CustomerNo']), axis=1
)
rows_to_remove = df_cancelled_orders_removed_tuples.isin(df_merged_tuples)
df_cancelled_orders_removed = df_cancelled_orders_removed[~rows_to_remove]

print(f"shape after cancelled orders are removed (step 2): {df_cancelled_orders_removed.shape}")

df_cancelled_orders_removed.sort_values('Quantity', ascending=False)

shape before cancelled orders are removed: (531095, 8)
shape after cancelled orders are removed (step 1): (522601, 8)
shape after cancelled orders are removed (step 2): (517349, 8)


,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
41367,578841,1574640000,84826,Asstd Design 3d Paper Stickers,6.19,12540,13256.0,United Kingdom
117790,573008,1572134400,84077,World War 2 Gliders Asstd Designs,10.47,4800,12901.0,United Kingdom
331834,554868,1558915200,22197,Popcorn Holder,10.99,4300,13135.0,United Kingdom
438495,544612,1550793600,22053,Empire Design Rosette,11.09,3906,18087.0,United Kingdom
269982,560599,1563494400,18007,Essential Balm 35g Tin In Envelope,10.31,3186,14609.0,United Kingdom
...,...,...,...,...,...,...,...,...
536316,536569,1543622400,22635,Childs Breakfast Set Dolly Girl,20.45,1,16274.0,United Kingdom
536315,536569,1543622400,22622,Box Of Vintage Alphabet Blocks,20.45,1,16274.0,United Kingdom
536312,536544,1543622400,22968,Rose Cottage Keepsake Box,20.45,1,17544.0,United Kingdom
536308,536530,1543622400,22120,Welcome Wooden Block Letters,20.45,1,17905.0,United Kingdom


## 3. Modeling (Sequential Pattern Mining)

In [86]:
from itertools import combinations

df_transformed = df_cancelled_orders_removed

# CONFIG
num_customers = 10

# overall performance evaluation
overall_true_positives = 0
overall_false_positives = 0
overall_false_negatives = 0
overall_true_negatives = 0

for customer_no in df_transformed['CustomerNo'].unique()[:num_customers]:
    # Extract transactions of the customer
    customer_data = df_transformed[df_transformed['CustomerNo'] == customer_no]
    customer_data = customer_data.sort_values(['Date', 'TransactionNo'])

    transactions = customer_data.groupby('TransactionNo')['ProductNo'].apply(list).tolist()

    # Split train and test
    split_index = int(len(transactions) * 0.7)
    train_transactions = transactions[:split_index]
    test_transactions = transactions[split_index:]

    # Sequential pattern mining
    min_support = 4
    patterns = list(seqmining.freq_seq_enum(train_transactions, min_support))

    # Performance evaluation for the current customer
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0

    tracker = {}
    for pattern, _ in patterns:
        if len(pattern) < 2:
            continue

        for product1, product2 in combinations(pattern, 2):
            if f"{product1}:{product2}" in tracker:
                continue
            else:
                tracker[f"{product1}:{product2}"] = True

            first_index = None
            second_index = None
            is_false_positive = True
            for idx, transaction in enumerate(test_transactions):
                if product1 in transaction and first_index is None:
                    first_index = idx

                if product2 in transaction and (second_index is None or idx > second_index):
                    second_index = idx

                if first_index and second_index and first_index < second_index:
                    true_positives += 1
                    is_false_positive = True
                    break

            if is_false_positive:
                false_positives += 1

        # TODO: true_negative and false_negative

    overall_true_positives += true_positives
    overall_false_positives += false_positives
    overall_false_negatives += false_negatives
    overall_true_negatives += true_negatives

# TODO: incorporate true_negative and false_negative
print(f"accuracy: {(100 * overall_true_positives) / (overall_true_positives + overall_false_positives):.2f}%")

3
12
accuracy: 20.00%


In [87]:
precision = overall_true_positives / (overall_true_positives + overall_false_positives) if (
                                                                                                   overall_true_positives + overall_false_positives) > 0 else 0
recall = overall_true_positives / (overall_true_positives + overall_false_negatives) if (
                                                                                                overall_true_positives + overall_false_negatives) > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (overall_true_positives + overall_true_negatives) / (
        overall_true_positives + overall_true_negatives + overall_false_positives + overall_false_negatives)

metrics = {
    "Metric": ["True Positives", "False Positives", "False Negatives", "True Negatives",
               "Precision", "Recall", "F1 Score", "Accuracy"],
    "Value": [overall_true_positives, overall_false_positives, overall_false_negatives, overall_true_negatives,
              f"{precision:.2f}", f"{recall:.2f}", f"{f1_score:.2f}", f"{accuracy:.2f}"]
}

pd.DataFrame(metrics)

,Metric,Value
0,True Positives,3
1,False Positives,12
2,False Negatives,0
3,True Negatives,0
4,Precision,0.20
5,Recall,1.00
6,F1 Score,0.33
7,Accuracy,0.20
